# Black Jack

Black Jack, ou 21 no Brasil, é um jogo de cartas que possui algumas variações. Para saber mais sobre as regras e as variações do jogo, acesse: https://pt.wikipedia.org/wiki/Blackjack

Nesse script foi criada uma versão simplificada do jogo em que temos apenas 1 jogador e o dealer. Como regra, se o jogador ou o dealer conseguir exatamente 21 pontos ele vence automaticamente. Se tiver mais de 21 perde automaticamente. O dealer sempre continuará comprando até atingir um mínimo de 17 pontos. Caso o jogador opte por não comprar mais cartas e se nem ele nem o dealer não tiverem atingido 21 pontos, o jogador pode "bater" para comparar sua pontuação com a do dealer. Nesse caso, quem tiver a pontuação mais alta, vence.

As cartas numéricas tem valor igual a seu número (rank). O 5 de copas vale 5 pontos, por exemplo. O valete, rei e rainha valem 10 pontos cada um. Já o Az pode valer 1 ou 11 pontos, dependendo de quantos pontos o jogador/dealer tem no momento em que comprou um dos 4 Azes.

Nesse script, é usado apenas 1 baralho que contém cartas numéricas, rei, rainha, valete e Az, totalizando 52 cartas.

O intuito desse script é exercitar a programação orientada a objeto (sigla POO em português ou OOP em inglês) através de um jogo simples e divertido. Esse foi meu segundo projeto que fiz sem ajuda externa. Por isso decidi não modificar o código original antes de postá-lo aqui, para poder mostrar e ver por conta própria, com os demais projetos que postarei aqui, a evolução dos meus scripts.

In [1]:
import random #Dessa biblioteca vamos usar o comando shuffle() para embaralhar o deck
from IPython.display import clear_output #Dessa biblioteca vamos importar o comando que limpa a tela de saída

#Definição dos naipes, tipos de cartas e seu valor correspondente
naipes = ('Copas','Paus','Espadas','Ouros')
ranks = ('Dois','Três','Quatro','Cinco','Seis','Sete','Oito','Nove','Dez','Jack','Queen','King','Ace')
valores = {'Dois':2, 'Três':3, 'Quatro':4, 'Cinco':5, 'Seis':6, 'Sete':7, 'Oito':8, 'Nove':9, 'Dez':10, 'Jack':10,
         'Queen':10, 'King':10, 'Ace':11}


#Variável que dirá se é a vez do player (True) ou não (False). O player sempre começa jogando
playing = True 

#Controla se o jogo vai continuar rodando (True) ou não (False)
jogo = True 

In [2]:
class Carta:
    """Essa classe contém os dados de uma carta como naipe, rank e valor numérico desse rank.
    Além disso, caso uma carta específica precise ser printada, ela aparece no formato:
    *rank* de *naipe*
    """
    def __init__(self,naipes,ranks):
        self.naipes = naipes
        self.ranks = ranks
        self.valores = valores[ranks]
    
    def __str__(self):
        return self.ranks + " de " + self.naipes

In [3]:
class Deck:
    """Essa é a classe do deck. Ela precisa combinar todos os dados das cartas para formar o deck
    de 52 cartas. Para cada naipe é criada uma carta com um rank. Ou seja, primeiro serão
    criadas todas as cartas de coração, depois todas as cartas de Paus e assim por diante.
    Para em baralhar o deck, temos um método especifico pra isso (shuffle, abaixo).
    Além disso, o ato de retirar uma carta do deck está associada à essa classe no método comprar.
    """
    def __init__(self):
        self.deck = []  # O deck começa vazio. Logo a seguir temos a criação das cartas que vamos colocar nesse deck
        for naipe in naipes: #para cada naipe em naipes:
            for rank in ranks: #e para cada rank em ranks:
                carta_criada = Carta(naipe,rank) #cria a carta
                self.deck.append(carta_criada) #coloca a carta criada no deck
    
    def __str__(self):
        for c in range(52): #pra cada carta dentro do baralho, que tem 52 cartas:
            print(self.deck[c]) #printa essa carta. ou seja, mostra as 52 cartas em sequencia

    def embaralhar(self):
        random.shuffle(self.deck) #embaralha o deck
        
    def distribuir(self):
        return self.deck.pop(0) #tira a carta do topo do deck

In [4]:
class Hand:
    """Essa classe é responsável por "segurar" as cartas compradas, mostrar o valor da pontuação atual
    do jogador e mostrar quantos aces o jogador tem.
    """
    def __init__(self):
        self.cartas = []  #A mão começa vazia, assim como o deck
        self.valor = 0   #A pontuação inicial da mão é zero
        self.aces = 0    #Como a mão começa vazia, não há aces inicialmente
    
    def add_card(self,carta): #adiciona uma carta à mao
        self.cartas.append(carta)   #A carta que for passada na chamada desse método é adicionada à mão
        if carta.ranks != 'Ace':   #se essa carta não for um ace:
            self.valor += valores[carta.ranks] # o valor dela é adicionado ao valor atual da mão
    
    def ajustar_ace(self): #ajusta o valor do ace
        lista = [] #lista que vai ter as cartas traduzidas para texto (o tipo inicial é "class")
        for c in range(len(self.cartas)): #para cada carta na mão:
            lista.append(str(self.cartas[c])) #essa carta vai ser convertida em texto e adicionada a lista de cartas
        #Aqui verificamos se a ultima carta da mão (ou seja, a que acabou de ser comprada) é um ace
        if lista[-1] == 'Ace de Copas' or lista[-1] =='Ace de Paus' or lista[-1]=='Ace de Ouros' or lista[-1]=='Ace de Espadas': #se tiver um ace na mão
            self.aces += 1 #se sim, acrescentamos 1 a contagem de aces na mão
            if self.valor > 10: #ainda se sim, caso o jogador tenha mais de 10 pontos:
                self.valor += 1 #o ace vai valer apenas 1 ponto
            else: #senão, ou seja, se o jogador tiver 10 pontos ou menos:
                self.valor += 11 #o ace vale 11 pontos
    
    def discartar_mao(self): #discarta todas as cartas da mão e zera a pontuação
        self.cartas = []
        self.valor = 0

In [5]:
class Fichas:
    """Essa classe representa as fichas que serão apostadas pelo player.
    Podemos verificar o valor da aposta e o total de fichas. Os cálculos
    relativos ao número de fichas, como quando o player ganha ou perde, ou 
    quando quer adicionar fichas, também estão aqui.
    """
    def __init__(self):
        self.total = 100  #O jogador sempre começa com 100 fichas
        self.aposta = 0   #O valor da aposta é escolhido pelo jogador
        
    def vence_aposta(self):
        self.total += self.aposta #o valor da aposta é adicionado ao valor total
    
    def perde_aposta(self):
        self.total -= self.aposta #o valor da aposta é subtraído do valor total
        
    def add_fichas(self):
        num = int(input('Digite quantas fichas quer adicionar: '))
        self.total += num #adiciona o valor digitado ao total

In [6]:
def definir_aposta(player):
    """Essa função é responsável por pegar o valor da aposta do player e 
    jogar esse valor no metodo aposta dentro da classe Fichas, ou seja,
    essa função define o número de fichas que o player quer apostar.
    """
    while True: #o bloco abaixo vai ser executado pra sempre, a menos que a palavra break seja executada
        try: #vamos fazer um teste (testar se o valor digitado é um número inteiro):
            valor_aposta = int(input('Digite quantas fichas quer apostar: ')) #primeiro o player aposta
            if player.total >= valor_aposta and player.total > 0: #se ele apostar menos fichas do que ele tem:
                player.aposta = valor_aposta #vamos registrar o valor da aposta
                break #e assim o loop acaba
            else: #senao:
                print(f'Você não tem fichas suficientes! Quantidade atual de fichas = {player.total}.\n')
                p = input('Deseja adicionar fichas (S ou N)? ')
                p.upper()
                while p != 'S' and p != 'N':
                    p = input('Digite apenas S ou N: \n\n')
                    p.upper()
                if p == 'S':
                    player.add_fichas()
        except: #caso o teste dê errado, ou seja, se for digitado qualquer coisa que nao seja um numero inteiro:
            print('Digite apenas valores inteiros!\n')

In [7]:
def comprar(deck,hand):
    """Essa função compra uma carta. deck é a classe Deck
    e hand é a classe Hand.
    """
    #usamos o método da classe Hand de adicionar uma carta à mão. Para isso precisamos indicar qual é essa carta.
    #assim usamos o método da classe Deck que retira a primeira carta do deck.
    #ou seja, retiramos a carta do deck e a colocamos na mão que for passada como argumento, seja do dealer ou do player.
    hand.add_card(deck.distribuir())

In [8]:
def compra_ou_passa(deck,hand):
    """Essa função define se é a vez do player atraves da variável playing e também
    verifica se o player quer comprar uma carta. Se ele quiser, a carta é comprada e se não
    a variável playing muda de estado indicando que é a vez da máquina. deck é a classe
    Deck e hand é a classe Hand.
    """
    global playing  #se for True, é a vez do player, se for False e a vez da máquina
    
    v = input('Deseja comprar uma carta? (Digite S ou N): ') #faz a pergunta e armazena a resposta
    v = v.upper() #faz a resposta ficar em letra maiuscula
    while v != 'S' and v != 'N': #enquanto a resposta do player não for S ou N (maiúsculo ou minúsculo):
        v = input('Digite apenas S para sim ou N para não: ') #ele vai precisar digitar de novo
        v = v.upper() #novamente a resposta dele será convertida para letra maiúscula
    if v == 'S': #se o player digitar s, ou seja, quiser comprar uma carta:
        comprar(deck,hand) #a carta é comprada e colocada na mão
    else: #se nao, ou seja, se o player quiser passar a vez:
        playing = False #como mencionado acima, se essa variável for false significa que é a vez da maquina

In [9]:
def mostrar_algumas(player,dealer):
    """Essa função mostra todas as cartas do player e oculta apenas a primeira carta
    do dealer. Mostra também a pontuação do player e do dealer de acordo com as cartas
    mostradas. player será a variável correspondente a mão do player assim como a
    dealer será a variável correspondente a mão do dealer.
    """
    #mostra as cartas e a pontuação do player
    print("\n\n__________________________________________________________________________\nCARTAS DO PLAYER:")
    for carta in range(len(player.cartas)): #para cada carta a mao do player:
        print(player.cartas[carta],end="      ") #vamos imprimir essas cartas
    print(f'          VALOR TOTAL: {player.valor} pontos.') #e mostrar o valor total delas nesse momento
    
    #mostra as cartas e a pontuação do dealer
    print("\n\nCARTAS DO DEALER:")
    for card in range(len(dealer.cartas)): #para cada carta na mao do dealer:
        if card == 0: #a primeira carta na mão do dealer:
            print("-----------------",end="      ") #é ocultada.   
        #as linhas 20, 21, 22 e 23 são apenas para ocultar um possível erro de indexação.
        #esse erro nao interfere na execução de nenhuma parte do programa.
        try:
            print(dealer.cartas[card+1],end="      ")
        except IndexError:
            pass
    #imprime o valor das cartas do dealer que estão sendo MOSTRADAS, ou seja, nao conta o valor da 1º carta
    print(f'          VALOR TOTAL: {dealer.valor-valores[dealer.cartas[0].ranks]} pontos.\n____________________________________________________________________') 
      
def mostrar_todas(player,dealer):
    """Essa função mostra todas as cartas de ambos os jogadores e a pontuação
    final de cada um. player será a variável correspondente a mão do player assim
    como dealer será a variável correspondente a mão do dealer.
    """
    #mostra todas as cartas e pontuação total do player
    print('\n\n\nRESULTADO FINAL DAS CARTAS DE AMBOS OS JOGADORES:')
    print("\n__________________________________________________________________________\nCARTAS DO PLAYER:")
    for carta in range(len(player.cartas)):
        print(player.cartas[carta],end="      ")
    print(f'          VALOR TOTAL: {player.valor} pontos.')
    
    #mostra todas as cartas e pontuação total do dealer
    print("\n\n\nCARTAS DO DEALER:")
    for card in range(len(dealer.cartas)):
        print(dealer.cartas[card],end="      ")
    print(f'          VALOR TOTAL: {dealer.valor} pontos.\n____________________________________________________________________')

In [10]:
def player_estoura(player,dealer,ficha):
    """Essa função termina o jogo caso a pontuação do player
    seja superior a 21 pontos, ou seja, ele estoura a pontuação.
    as variáveis player e dealer são a classe Hand associada a cada jogador.
    A variável ficha é a classe Ficha.
    """
    global jogo #permite alterar a variável jogo do inicio do código
    if player.valor > 21: #se a pontuação total do player for maior do que 21:
        print('\n\n\n\nPlayer estourou a pontuação! DEALER WINS!!!') #mostra essa mensagem
        ficha.perde_aposta() #é executado o método que reduz a quantidade de fichas que o player tem
        print(f"Total de fichas = {ficha.total}.") #mostra quantas fichas o player tem no momento
        jogo = False
    
def player_vence(player,dealer,ficha):
    """Essa função termina o jogo caso a pontuação do player
    seja igual a 21 pontos, ou seja, ele atinge a pontuação máxima.
    as variáveis player e dealer são a classe Hand associada a cada jogador.
    A variável ficha é a classe Ficha.
    """
    global jogo #permite alterar a variável jogo do inicio do código
    if player.valor == 21: #se a pontuação do player for exatamente igual a 21:
        print('\n\n\n\nPlayer atingiu a pontuação máxima! PLAYER WINS!!!') #mostra essa mensagem
        ficha.vence_aposta() #o player recebe mais fichas
        print(f"Total de fichas = {ficha.total}.") #mostra a quantidade atual de fichas
        jogo = False

def dealer_estoura(player,dealer,ficha):
    """Essa função termina o jogo caso a pontuação do dealer
    seja superior a 21 pontos, ou seja, ele estoura a pontuação.
    As variáveis player e dealer são a classe Hand associada a cada jogador.
    A variável ficha é a classe Ficha.
    """
    global jogo #permite alterar a variável jogo do inicio do código
    if dealer.valor > 21:#se a pontuação total do dealer for maior do que 21:
        print('\n\n\n\nDealer estourou a pontuação! PLAYER WINS!!!')#mostra essa mensagem
        ficha.vence_aposta() #player ganha fichas
        print(f"Total de fichas = {ficha.total}.")#mostra a quantidade atual de fichas
        jogo = False
    
def dealer_vence(player,dealer,ficha):
    """Essa função termina o jogo caso a pontuação do dealer
    seja igual a 21 pontos, ou seja, ele atinge a pontuação máxima.
    as variáveis player e dealer são a classe Hand associada a cada jogador.
    A variável ficha é a classe Ficha.
    """
    global jogo #permite alterar a variável jogo do inicio do código
    if dealer.valor == 21:#se a pontuação do dealer for exatamente igual a 21:
        print('\n\n\n\nDealer atingiu a pontuação máxima! DEALER WINS!!!')#mostra essa mensagem
        ficha.perde_aposta()#player perde fichas
        print(f"Total de fichas = {ficha.total}.")#mostra a quantidade atual de fichas
        jogo = False
    
def bater(player,dealer,ficha,deck):
    """Essa função é usada quando o player quer comparar sua pontuação
    com a do dealer. Quem tiver a maior pontuação vence. Porém, se houver
    um empate, ambos os jogadores discartam sua mão e compram uma carta.
    Quem comprar a carta de maior valor vence. Esse processo se repete até
    que alguem vença a partida, caso as cartas compradas tenham o mesmo valor.
    As variáveis player e dealer são a classe Hand associada a cada jogador.
    A variável ficha é a classe Ficha. deck é a classe deck
    """ 
    global jogo #permite alterar a variável jogo do inicio do código
    if player.valor > dealer.valor: #se o player tem mais pontos que o dealer:
        print('\n\n\n\nPLAYER WINS!!!') #mostra essa mensagem
        ficha.vence_aposta() #player ganha fichas
        print(f"Total de fichas = {ficha.total}.")#mostra a quantidade atual de fichas
        jogo = False
    elif player.valor == dealer.valor: #se o player e o dealer tiverem a mesma pontuação total:
        player.discartar_mao() #player descarta sua mao e tem sua pontuação zerada
        dealer.discartar_mao() #dealer descarta sua mao e tem sua pontuação zerada
        while True: #enquanto uma condição de vitoria nao for definida:
            player.add_card(deck.distribuir()) #player compra uma carta
            dealer.add_card(deck.distribuir()) #dealer compra uma carta
            
            #as cartas que acabaram de ser compradas sao mostradas
            print(f"Player: {player.cartas[-1]}\n\nDealer: {dealer.cartas[-1]}")
            
            #se o valor da carta que o player comprou for maior do que o valor da carta que o dealer comprou:
            if valores[player.cartas[-1].ranks] > valores[dealer.cartas[-1].ranks]:
                print("\n\n\n\nPLAYER WINS!!!")
                ficha.vence_aposta() #player ganha fichas
                print(f"Total de fichas = {ficha.total}.")#mostra a quantidade atual de fichas
                jogo = False
                break #saimos do loop ja que definimos um vencedor
            #se o valor da carta que o player comprou for menor do que o valor da carta que o dealer comprou:
            elif valores[player.cartas[-1].ranks] < valores[dealer.cartas[-1].ranks]:
                print('\n\n\n\nDEALER WINS!!!')
                ficha.perde_aposta()#player perde fichas
                print(f"Total de fichas = {ficha.total}.")#mostra a quantidade atual de fichas
                jogo = False
                break #saimos do loop ja que definimos um vencedor
    else:
        print('\n\n\n\nDEALER WINS!!!')
        ficha.perde_aposta()#player perde fichas
        print(f"Total de fichas = {ficha.total}.")#mostra a quantidade atual de fichas
        jogo = False

In [11]:
def retry():
    global jogo
    x = input('\n\nDeseja jogar novamente (S ou N)? ')
    x.upper()
    while x != 'S' and x!='N':
        x = input('Responda apenas com S ou N: ')
        x.upper()
    if x == 'S':
        jogo = True
    else:
        jogo = False
        print("GAME OVER!")

In [12]:
ficha = Fichas() #cria as fichas do player
while jogo: #enquanto o jogo estiver rodando:
    print("Bem-vindo ao Black Jack!") #abertura
    novo_deck = Deck() # cria o deck
    novo_deck.embaralhar() #embaralha o deck
    mao_player = Hand() #cria a mao do player
    mao_dealer = Hand() #cria a mao do dealer
    for a in range(2): #Por duas vezes:
        comprar(novo_deck,mao_player) #o player compra uma carta
        mao_player.ajustar_ace() #ajusta o valor dos aces
        comprar(novo_deck,mao_dealer) #o dealer compra uma carta
        mao_dealer.ajustar_ace() #ajusta o valor dos aces
    definir_aposta(ficha) #player define sua aposta (lembrando que ele começa com 100 fichas)
    mostrar_algumas(mao_player,mao_dealer) #todas as cartas do player e todas menos 1 do dealer são mostradas
    dealer_vence(mao_player,mao_dealer,ficha) #verificamos se o player ganha na primeira rodada
    player_vence(mao_player,mao_dealer,ficha) #verificamos se o player ganha na primeira rodada
        
    #depois de termos criado as cartas, o deck, embaralhar o deck, as duas maos, as fichas, distribuir as cartas
    #ajustando os aces, definir a aposta, mostrar as duas mãos e verificar se ja existe um vencedor
    #o primeiro turno é dado ao player.
    
    while playing and jogo: #enquanto for a vez do player:
        compra_ou_passa(novo_deck,mao_player) #é perguntado se o player quer comprar ou passar
        mao_player.ajustar_ace() #ajusta o valor dos aces
        if len(mao_player.cartas) > 2: #Se o player tiver mais de duas cartas:
            mostrar_algumas(mao_player,mao_dealer) #as cartas são mostradas novamente
        player_vence(mao_player,mao_dealer,ficha) #verificação se o player atingiu 21 pontos
        if mao_player.valor > 21: #se o player estourar:
            player_estoura(mao_player,mao_dealer,ficha) #player perde
            break
            
    while mao_dealer.valor <= 17 and jogo: #enquanto o dealer tiver menos de 17 pontos:
        comprar(novo_deck,mao_dealer) #o dealer compra cartas
        mao_dealer.ajustar_ace() #ajusta o valor dos aces
        dealer_vence(mao_player,mao_dealer,ficha) #verificamos se o dealer atingiu 21 pontos
        dealer_estoura(mao_player,mao_dealer,ficha) #verificamos se o dealer estourou a pontuação
    
    if jogo: #se ninguem tiver ganhado ainda, ou seja, jogo = True:
        clear_output() #limpa a tela de saída
        bater(mao_player,mao_dealer,ficha,novo_deck) #executamos a função bater que vai definir um vencedor
        
    mostrar_todas(mao_player,mao_dealer) #todas as cartas e pontuação final são mostradas
    
    retry() #pergunta se queremos jogar novamente
    clear_output()